# Test Classifier

In [1]:
model_type = 'transformer'

In [2]:
import sys
sys.path.append('../../')

In [3]:
%cd ../../

C:\Users\fillj\Documents\PyCharm Projects\predicting_building_types_functions


In [4]:
import torch
import json
import torch.nn as nn
import torch_geometric

import sample.dataset.gnn_dataset_modeling as dsm
import sample.training.fetch_model as fm
import sample.training.eval as ev
import torch_geometric.loader as loader
import sample.training.split_dataset as sd
import sample.training.aux_fcnn as ax
import sample.util.feature_names as fn

In [5]:
model_name = f'final_{model_type}'

In [6]:
with open(f'sample/training/config/{model_type}.json', 'r') as json_file:
    config_dict = json.load(json_file)

In [7]:
class Config:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
config = Config(**config_dict)

In [8]:
torch_geometric.seed_everything(42)
path = f'./sample/dataset/pyg_ds'
dataset = dsm.GNNDatasetModeling(path, '')
dataset_train, dataset_val, dataset_test = sd.split_train_val_test(dataset)
dataset_train, dataset_val, dataset_test = sd.mark_overlapping_labels(dataset_train, dataset_val, dataset_test)
# For non-GNN-based models, flatten graph
if model_type in ['dt', 'rf', 'fcnn']:
    _, _, _, _, x_test, y_test = ax.preprocessing(dataset_train, dataset_val, dataset_test)
# Load data
if model_type in ['dt', 'rf']:
    dataloader_test = x_test, y_test
elif model_type == 'fcnn':
    dataloader_test = ax.load_data(x_test, y_test, batch_size=64, shuffle=False)
else:
    dataloader_val = loader.DataLoader(dataset_val, 64, shuffle=False)
    dataloader_test = loader.DataLoader(dataset_test, 64, shuffle=False)
# Determine device. Train on GPU if available (only used for NN models)
device = (
        'cuda'
        if torch.cuda.is_available()
        else 'mps'
        if torch.backends.mps.is_available()
        else 'cpu'
)
# Set loss function (only used for NN models)
loss_fn = nn.NLLLoss()
model = fm.fetch_model(config, model_name, model_type, device)
y_predict, _ = ev.evaluate(dataloader_test, model, device, loss_fn, True, None, model_type)

Accuracy score (all): 0.6988352745424293
Cohen's Kappa Coefficient (all): 0.5836373031213518
Matthew's Correlation Coefficient (all): 0.5899503153222679
Macro F1-score (all): 0.4039375665780174
Precision score (apartments): 0.8538461538461538
Recall score (apartments): 0.7816901408450704
F1-score (apartments): 0.8161764705882353
Precision score (detached): 0.45038167938931295
Recall score (detached): 0.855072463768116
F1-score (detached): 0.59
Precision score (semidetached): 0.0
Recall score (semidetached): 0.0
F1-score (semidetached): 0.0
Precision score (terrace): 0.6216216216216216
Recall score (terrace): 0.6216216216216216
F1-score (terrace): 0.6216216216216216
Precision score (industrial): 0.0
Recall score (industrial): 0.0
F1-score (industrial): 0.0
Precision score (commercial): 0.375
Recall score (commercial): 0.5294117647058824
F1-score (commercial): 0.4390243902439025
Precision score (public): 0.3333333333333333
Recall score (public): 0.4
F1-score (public): 0.3636363636363636


C:\Users\fillj\anaconda3\envs\geo_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fillj\anaconda3\envs\geo_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
